In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/ai agent trainning/train/reward')

Mounted at /content/drive


In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
import yaml

with open('../configs/base.yaml', 'r') as file:
    configs = yaml.safe_load(file)

with open('../configs/sft.yaml', 'r') as file:
    configs.update(yaml.safe_load(file))

In [ ]:
model_name = configs['model_name']
pretrain_model_path = f'../../lora_models/sft/{model_name}'

In [ ]:
if True:
    from unsloth import FastLanguageModel
    unsloth_model, unsloth_tokenizer = FastLanguageModel.from_pretrained(
        model_name = pretrain_model_path, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = configs['model']['max_seq_length'],
        dtype = configs['model']['dtype'],
        load_in_4bit = configs['model']['load_in_4bit'],
    )
    FastLanguageModel.for_inference(unsloth_model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: We'll be using `/tmp/unsloth_compiled_cache` for temporary Unsloth patches.
Standard import failed for UnslothORPOTrainer: No module named 'UnslothORPOTrainer'. Using tempfile instead!
==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

Unsloth 2025.4.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
from datasets import load_dataset
data_path = configs['data_path']
dataset = load_dataset(data_path, split = "train")

README.md:   0%|          | 0.00/358 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/85.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/111701 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import get_chat_template

unsloth_tokenizer = get_chat_template(
    unsloth_tokenizer,
    chat_template = configs['chat_template'],
)

def formatting_prompts_func(examples):
    convos = examples["conversation"]
    texts = [unsloth_tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/111701 [00:00<?, ? examples/s]

In [ ]:
def create_batch(dataset, batch_size=16):
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size]

batch = [x for x in create_batch(dataset)]
len(batch)

6982

In [ ]:
def get_user_prompt(text):
    return text.split('<|end_header_id|>')[2]\
                                        .replace('<|eot_id|>', '')\
                                        .replace('<|start_header_id|>assistant', '')\
                                        .strip()

In [ ]:
import json

def update_progress(i, outputs):
    with open('respone.json', 'r') as f:
        data = json.load(f)

    data += outputs

    with open('respone.json', 'w') as f:
        json.dump(data, f)

    with open('ok.txt', 'w') as f:
        f.write(str(i))

In [ ]:
max_new_tokens = 256
def generate(messages):
    inputs = unsloth_tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
        padding=True,
        truncation=True
    ).to("cuda")

    outputs = unsloth_model.generate(input_ids = inputs, max_new_tokens = max_new_tokens, use_cache = True,
                            temperature = 1.5, min_p = 0.1)
    outputs = [x.split('<|start_header_id|>assistant<|end_header_id|>')[-1].strip().replace('<|eot_id|>', '') for x in unsloth_tokenizer.batch_decode(outputs)]
    return outputs

In [ ]:
with open('ok.txt', 'r') as f:
    ok = int(f.read())
assert False, f"SUCCESS\n\nSố index đang là {ok}"

AssertionError: SUCCESS

Số index đang là 121

In [ ]:
from tqdm import tqdm

for i, batch_data in tqdm(enumerate(batch)):
    if i <= ok:
        continue
    texts = batch_data['text']
    respones = dict()

    messages = [[{"role": "user", "content": get_user_prompt(text)},] for text in texts]
    respones['messages'] = messages

    outputs = generate(messages)
    respones['outputs 1'] = outputs

    messages = [[{"role": "user", "content": f'{get_user_prompt(text)}. Hãy trình bày phản hồi dưới dạng liệt kê bằng dấu đầu dòng.'},] for text in texts]
    outputs = generate(messages)
    respones['outputs 2'] = outputs

    update_progress(i, [respones])

0it [00:00, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
195it [3:06:52, 150.55s/it]

In [3]:
import json

with open('../../data/reward/data.json', 'r') as f:
    data = json.load(f)
with open('../../data/reward/reward_data.json', 'r') as f:
    data += json.load(f)

In [5]:
with open('../../data/reward/health-care-reward.json', 'w') as f:
    json.dump(data, f)